In [14]:
from os.path import join
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from os.path import dirname,join
import sys

ROOT_DIR = "../"
sys.path.append("../")

from process_pipeline.labels import *

In [16]:
df = pd.read_parquet("../data/processed/dyconex_test/output/df_input_short.parquet")
df

,Unnamed: 0,batch,position,date,parameter,value,variable,process,value_norm,Year,Month,Day,Hour,Minute
0,1290,CTAO,440.0,2022-01-03 16:43:00,Pulsenergie,0.302853,las_11,Laser,0.302853,2022.0,1.0,3.0,16.0,43.0
1,1291,CTAO,440.0,2022-01-03 16:54:00,Pulsenergie,0.303713,las_11,Laser,0.303713,2022.0,1.0,3.0,16.0,54.0
2,1292,CTAO,440.0,2022-01-03 16:59:00,Pulsenergie,0.303884,las_11,Laser,0.303884,2022.0,1.0,3.0,16.0,59.0
3,1293,CTAO,440.0,2022-01-03 17:07:00,Pulsenergie,0.303025,las_11,Laser,0.303025,2022.0,1.0,3.0,17.0,7.0
4,1294,CTAO,440.0,2022-01-03 17:14:00,Pulsenergie,0.302509,las_11,Laser,0.302509,2022.0,1.0,3.0,17.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11277626,21242497,CXHN,290.0,2023-12-06 09:03:03,UCL_Spüle Bakterienbefall-M5.4,0.964319,mic_75,Microetch,0.964320,2023.0,12.0,6.0,9.0,3.0
11277627,21242498,CXHN,680.0,2024-01-22 16:52:12,UCL_Spüle Bakterienbefall-M5.4,0.964319,mic_75,Microetch,0.964320,2024.0,1.0,22.0,16.0,52.0
11277628,21242499,CXHS,680.0,2024-01-16 19:23:12,UCL_Spüle Bakterienbefall-M5.4,0.964319,mic_75,Microetch,0.964320,2024.0,1.0,16.0,19.0,23.0
11277629,21242500,CXOF,290.0,2023-12-08 11:04:46,UCL_Spüle Bakterienbefall-M5.4,0.964319,mic_75,Microetch,0.964320,2023.0,12.0,8.0,11.0,4.0


In [ ]:

DATA_DIR = join(ROOT_DIR,"data")
OUT_DIR = join(DATA_DIR,"output")
INTERMEDIATE_DIR = join(DATA_DIR,"intermediate")

In [85]:
df_tpt = pd.read_csv(join(OUT_DIR,templates_filename))
df_tpt.head()

,Unnamed: 0,PaPos,Variable,AbsPos,Process,SAP,Version
0,0,40,las_11,0,Laser,426816,C
1,1,40,las_12,0,Laser,426816,C
2,2,40,las_13,0,Laser,426816,C
3,3,40,las_14,0,Laser,426816,C
4,4,40,las_15,0,Laser,426816,C


In [86]:
for design in df_tpt[templates_design_label].unique():
    df_sel = df_tpt.set_index(templates_design_label).loc[design]
    for version in df_sel[templates_version_label].unique():
        length = len(df_sel[df_sel[templates_version_label]==version])
        print(f"{design} {version} = {length}")
    del df_sel

426816 C = 1568
426816 D = 2677
436425 H = 629
436425 I = 1532
436425 J = 2785
449669 B = 484
449669 C = 2894
453828 B = 3393


In [87]:
df_pro = pd.read_csv(join(INTERMEDIATE_DIR,input_raw_filename))
df_lev = pd.read_csv(join(INTERMEDIATE_DIR,input_leveled_filename))

In [88]:
id_sel = "50408A"
df_lev_sel = df_lev.set_index("id").loc[id_sel]
df_pro_sel = df_pro.set_index("id").loc[id_sel]
df_lev_sel.head()


,Unnamed: 0,PaPos,Variable,AbsPos,Process,SAP,Version,WA,Value,Time,Process.1,Given
id,,,,,,,,,,,,
50408A,14055094,700,las_11,0.0,Laser,453828,B,CXCF,NaN,NaN,NaN,0
50408A,14055095,700,las_12,0.0,Laser,453828,B,CXCF,NaN,NaN,NaN,0
50408A,14055096,700,las_13,0.0,Laser,453828,B,CXCF,NaN,NaN,NaN,0
50408A,14055097,700,las_14,0.0,Laser,453828,B,CXCF,NaN,NaN,NaN,0
50408A,14055098,700,las_15,0.0,Laser,453828,B,CXCF,NaN,NaN,NaN,0


In [89]:
missing_steps = df_lev_sel[df_lev_sel["Given"]==0]["PaPos"].unique()
given_steps = df_lev_sel[df_lev_sel["Given"]==1]["PaPos"].unique()

count = 0
for step in given_steps:
    if step in missing_steps:
        print(step)
        count += 1

print(f"{count/len(given_steps)*100}% of given are also in missing")

40
60
720
90
260
300
470
510
750
250
270
460
480
81.25% of given are also in missing


In [90]:
df_pro_sel.drop_duplicates(subset=["PaPos","Process"])

,Unnamed: 0,Value,Time,Variable,Process,WA,PaPos,Version,SAP
id,,,,,,,,,
50408A,0,0.803854,7/22/23 11:03 AM,las_22,Laser,CXCF,40,B,453828
50408A,0,0.166667,9/18/23 11:53 AM,pla_7,Plasma,CXCF,60,B,453828
50408A,0,0.255494,8/8/23 5:00 AM,gal_9,Galvanic,CXCF,90,B,453828
50408A,0,0.545454,8/14/23 2:48 PM,mul_10,Multibond,CXCF,200,B,453828
50408A,0,0.803854,7/22/23 11:03 AM,las_22,Laser,CXCF,250,B,453828
50408A,0,0.803854,7/22/23 11:03 AM,las_22,Laser,CXCF,260,B,453828
50408A,0,0.166667,9/18/23 11:53 AM,pla_7,Plasma,CXCF,270,B,453828
50408A,0,0.255494,8/8/23 5:00 AM,gal_9,Galvanic,CXCF,300,B,453828
50408A,0,0.803854,7/22/23 11:03 AM,las_22,Laser,CXCF,460,B,453828


In [91]:
len_lev = len(df_lev_sel[df_lev_sel["Given"]==1])
len_orig = len(df_pro_sel)
print(len_lev,len_orig)

1281 1287


In [92]:
df1 = pd.DataFrame.from_dict({"A": [1,2,3],
                            "B": ["a","b","c"],
                            "Values" : [10,20,30]})

df2 = pd.DataFrame.from_dict({"A": [1,3],
                            "B": ["a","c"],
                            "Values" : [11,33]})

df1 = df1.set_index(["A","B"])
df2 = df2.set_index(["A","B"])

df3 = pd.concat([df1,df2],axis = 1,keys=["A","B"])
df3


,,A,B
,,Values,Values
A,B,,
1,a,10,11.0
2,b,20,NaN
3,c,30,33.0


In [93]:
df1

,,Values
A,B,
1,a,10
2,b,20
3,c,30


In [94]:
df=pd.read_csv("../data/test/_df.csv")
df_templ=pd.read_csv("../data/test/template.csv")
df_templ = df_templ.set_index(["PaPos","Process","Variable"])
df = df.set_index(["PaPos","Process","Variable"])

In [98]:
print(f"len template: {len(df_templ)}")
print(f"len df: {len(df)}")
df_templ.head()

len template: 3393
len df: 1287


Version  AbsPos
PaPos Process Variable                
40    Laser   las_11         B       9
              las_12         B       9
              las_13         B       9
              las_14         B       9
              las_15         B       9

In [99]:
df.head()

SAP Version    WA      id  Unnamed: 0     Value  \
PaPos Process Variable                                                       
40    Laser   las_22    453828       B  CXCF  50408A           0  0.803854   
              las_23    453828       B  CXCF  50408A           1  0.773797   
              las_28    453828       B  CXCF  50408A           2  0.465686   
              las_29    453828       B  CXCF  50408A           3  0.775458   
              las_30    453828       B  CXCF  50408A           4  0.005767   

                                    Time  
PaPos Process Variable                    
40    Laser   las_22    7/22/23 11:03 AM  
              las_23    7/22/23 11:03 AM  
              las_28    7/22/23 11:03 AM  
              las_29    7/22/23 11:03 AM  
              las_30    7/22/23 11:03 AM

In [101]:
df_lev_temp = pd.concat([df_templ,df],axis=1)
print(f"len concat: {len(df_lev_temp)}")
df_lev_temp.head()

len concat: 3559


Version  AbsPos  SAP Version   WA   id  Unnamed: 0  \
PaPos Process Variable                                                      
40    Laser   las_11         B     9.0  NaN     NaN  NaN  NaN         NaN   
              las_12         B     9.0  NaN     NaN  NaN  NaN         NaN   
              las_13         B     9.0  NaN     NaN  NaN  NaN         NaN   
              las_14         B     9.0  NaN     NaN  NaN  NaN         NaN   
              las_15         B     9.0  NaN     NaN  NaN  NaN         NaN   

                        Value Time  
PaPos Process Variable              
40    Laser   las_11      NaN  NaN  
              las_12      NaN  NaN  
              las_13      NaN  NaN  
              las_14      NaN  NaN  
              las_15      NaN  NaN